## XGBoost

Chegamos ao nosso último método de ensemble, o XGBoost (e**X**treme **G**radient **Boost**ing).

Este método nada mais é que um gradient boosting, mas com algumas importantes modificações que lhe conferem o título de "extreme"! Em particular, duas alterações merecem destaque:

- A adição de procedimentos de regularização (L1 e L2!), o que melhora consideravelmente sua capacidade de generalização;

- A utilização de derivadas de segunda ordem (Hessiano) para o procedimento de gradiente.

Para quem quiser se aventurar mais, sugiro algumas boas leituras:

- [Este](https://shirinsplayground.netlify.app/2018/11/ml_basics_gbm/), explica bem as particularidades do XGBoost, além de dar uma boa introdução ao gradient boosting (o código é em R, então pode ignorar essa parte hehe);

- [Este](https://medium.com/analytics-vidhya/what-makes-xgboost-so-extreme-e1544a4433bb), introduz bem o método, enquanto enfativa suas particularidades, com alguns detalhes matemáticos;

- [Este](https://xgboost.readthedocs.io/en/latest/tutorials/model.html), da própria documentação da biblioteca, traz uma explicação legal, e com alguns detalhes matemáticos;

- [Este](https://towardsdatascience.com/https-medium-com-vishalmorde-xgboost-algorithm-long-she-may-rein-edd9f99be63d), com uma discussão mais alto-nível (sem tantos detalhes) sobre o XGBoost e os motivos de seu sucesso.

Basicamente temos um GradientBoost Extreme:
- Quanto ao paralelismo, a construção sequêncial de arvores é paralelisada.
    - Temos o loop externo que enumera os nós folha de uma árvore e o segundo loop interno que calcula os recursos, o truque é primeiro preparar várias linhas de execução para cada folha e cada uma delas é disparada no momento do cálculo da hipotese.
- Poda da árvore: Após a expansão da árvore, pode ocorrer que os subconjuntos de nós folhas sejam muito pequenos e com grande efeito de overfitting e esse ajuste pode ser minizado, já que temos o critério de max_depth.
- Otimização de hardware.
- Já regulariza por Lasso e Ridge.
- Tratamento de valores faltantes, já "aprende" qual o melhor valor para adotar
- Aplica quantil ponderado nos dados.
- Validação cruzada integrado.

<img src='https://cdn-images-1.medium.com/max/1000/1*U72CpSTnJ-XTjCisJqCqLg.jpeg' width=600 height=1200/>

Alguns hiperparâmetros: ( https://xgboost.readthedocs.io/en/latest/parameter.html#general-parameters )
- learning_rate: taxa de aprendizado, valores menores de passo usado para evitar overfitting. O intervalo é [0,1].
- max_depth: determina quão profundamente cada árvore pode crescer durante qualquer rodada de reforço.
- subsample: porcentagem de amostras utilizadas por árvore. O baixo valor pode levar ao underfitting.
- colsample_bytree: porcentagem de recursos usados por árvore. Alto valor pode levar a overfitting.
- n_estimators: número de árvores que você deseja construir.
- objective: determina a função de perda a ser usada como reg:linear para problemas de regressão, reg:logistic para problemas de classificação com apenas decisão, binary:logistic para problemas de classificação com probabilidade.
- Regularização
    - gamma: controla se um determinado nó será dividido com base na redução esperada na perda após a divisão. Um valor mais alto leva a menos divisões. Suportado apenas para alunos baseados em árvore.
    - alpha: Regularização L1 nos pesos das folhas. Um valor grande leva a mais regularização.
    - lambda: Regularização L2 nos pesos das folhas e é mais suave do que a regularização L1.
- Validação Cruzada
    - num_boost_round: denota o número de árvores que você constrói (análogo a n_estimators)
    - metrics: informa as métricas de avaliação a serem observadas durante o CV
    - early_stopping_rounds: termina o treinamento do modelo antecipadamente se a métrica de hold-out ("rmse" em nosso caso) não melhorar em um determinado número de rodadas.

Infelizmente, o sklearn não tem o XGBoost implementado :(

Mas, felizmente, existe uma biblioteca que o implementou, de maneira totalmente integrada ao sklearn!!

A biblioteca é a [XGBoost](https://xgboost.readthedocs.io/en/latest/).

Para instalar a biblioteca, o de sempre: ( https://xgboost.readthedocs.io/en/latest/install.html#python )

`!pip install xgboost`

In [1]:
!pip install xgboost

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import classification_report

In [6]:
import xgboost as xgb

In [19]:
from sklearn.datasets import load_breast_cancer

In [20]:
df = load_breast_cancer(as_frame = True)["frame"]
df.head()

y_colum = 'target'

X = df.drop(columns=[y_colum])
y = df[y_colum]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [4]:
def pipe_preprocessor(path_dataset, y_colum):
    df = pd.read_csv(path_dataset, index_col=0)

    X = df.drop(columns=[y_colum])
    y = df[y_colum]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    pipe_features_num = Pipeline([
        ('input_num', SimpleImputer(strategy='mean')),
        ('std', StandardScaler())
    ])
    
    features_num = X_train.select_dtypes(include=np.number).columns.tolist()
    
    pipe_features_cat = Pipeline([
        ('input_cat', SimpleImputer(strategy='constant', fill_value='unknown')),
        ('ondehot', OneHotEncoder())
    ])
    
    features_cat = X_train.select_dtypes(exclude=np.number).columns.tolist()
    
    pre_processor = ColumnTransformer([
        ('transf_num', pipe_features_num, features_num),
        ('transf_cat', pipe_features_cat, features_cat)
    ])
    
    return pre_processor

In [22]:
def metricas_classificacao(estimator):
    
    # ============================================

    print("\nMétricas de avaliação de treino:")

    y_pred_train = estimator.predict(X_train)

    print(classification_report(y_train, y_pred_train))


    # ============================================

    print("\nMétricas de avaliação de teste:")

    y_pred_test = estimator.predict(X_test)


    print(classification_report(y_test, y_pred_test))

In [23]:
model = xgb.XGBClassifier()

In [24]:
model.fit(X_train, y_train)

[22:11:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [25]:
metricas_classificacao(model, X_train, y_train)

TypeError: metricas_classificacao() takes 1 positional argument but 3 were given

Aumentando um pouco a regularização:

- Boosting e estratégias evolucionárias na tarefa de regressao para a mineraçao de dados temporais
    - https://acervodigital.ufpr.br/handle/1884/3770?show=full
- XGBoosting visão geral
    - https://www.kdnuggets.com/2019/05/xgboost-algorithm.html

#### Exercício
Leia o material https://towardsdatascience.com/xgboost-fine-tune-and-optimize-your-model-23d996fab663  
Pratique o utilizar um GridSearch para ajudar os hiperparâmetros e relate um ajuste que melhorou o modelo.